# Dictionaries and sets

Hash tables are the engines behind Python’s high performance dicts.

"An object is hashable if it has a hash value which never changes during its lifetime (it needs a `__hash__()` method), and can be compared to other objects (it needs an `__eq__()` method). Hashable objects which compare equal must have the same hash value.""

**“All of Python’s immutable built-in objects are hashable, but not all hashable objects are immutable”**

In [1]:
my_name = "Carla"
hash(my_name)

5122140301632635949

In [4]:
l = [1, 2, 3]
hash(l)

TypeError: unhashable type: 'list'

## Different ways to create a dictionary

In [2]:
a = dict(one=1, two=2, three=3)

b = {'one': 1, 'two': 2, 'three': 3}

c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))  

d = dict([('two', 2), ('one', 1), ('three', 3)])  

e = dict({'three': 3, 'one': 1, 'two': 2}) 
assert a == b == c == d == e

In [3]:
### zip to generate dictionaries
keys = ['a', 'b', 'c']
values = [1, 2, 3]
d3 = dict(zip(keys, values))
d3

{'a': 1, 'b': 2, 'c': 3}

## dict comprehensions

In [4]:
DIAL_CODES = [
	(86, 'China'),
	(91, 'India'),
	(1, 'United States'),
	(62, 'Indonesia'),
	(55, 'Brazil'),
	(92, 'Pakistan'),
	(880, 'Bangladesh'),
	(234, 'Nigeria'),
	(7, 'Russia'),
	(81, 'Japan'),
]

In [5]:
country_code = {country: code for code, country in DIAL_CODES}

In [6]:
country_code

{'China': 86,
 'India': 91,
 'United States': 1,
 'Indonesia': 62,
 'Brazil': 55,
 'Pakistan': 92,
 'Bangladesh': 880,
 'Nigeria': 234,
 'Russia': 7,
 'Japan': 81}

In [7]:
{code: country.upper() for country, code in country_code.items() if code < 66}   

{1: 'UNITED STATES', 62: 'INDONESIA', 55: 'BRAZIL', 7: 'RUSSIA'}

# Handling missing keys

In [8]:
doggo = {'name': 'Lassie','breed': "border collie"}

In [9]:
doggo.nome

AttributeError: 'dict' object has no attribute 'nome'

In [32]:
doggo.get('nome', 'banana')

'banana'

## With `get`

In [31]:
"""Build an index mapping word -> list of occurrences"""

import sys
import re

WORD_RE = re.compile('\w+')
FILE_NAME = 'zen.txt'

index = {}

with open(FILE_NAME, encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            # this is ugly; coded like this to make a point
            occurrences = index.get(word, [])
            occurrences.append(location)
            index[word] = occurrences
        
# print in alphabetical order
for word in sorted(index, key=str.upper):
    print(word, index[word])

a [(17, 48), (18, 53)]
Although [(9, 1), (14, 1), (16, 1)]
ambiguity [(12, 16)]
and [(13, 23)]
are [(19, 12)]
aren [(8, 15)]
at [(14, 38)]
bad [(17, 50)]
be [(13, 14), (14, 27), (18, 50)]
beats [(9, 23)]
Beautiful [(1, 1)]
better [(1, 14), (2, 13), (3, 11), (4, 12), (5, 9), (6, 11), (15, 8), (16, 25)]
break [(8, 40)]
cases [(8, 9)]
complex [(3, 23)]
Complex [(4, 1)]
complicated [(4, 24)]
counts [(7, 13)]
dense [(6, 23)]
do [(13, 64), (19, 48)]
Dutch [(14, 61)]
easy [(18, 26)]
enough [(8, 30)]
Errors [(10, 1)]
explain [(17, 34), (18, 34)]
Explicit [(2, 1)]
explicitly [(11, 8)]
face [(12, 8)]
first [(14, 41)]
Flat [(5, 1)]
good [(18, 55)]
great [(19, 28)]
guess [(12, 52)]
hard [(17, 26)]
honking [(19, 20)]
idea [(17, 54), (18, 60), (19, 34)]
If [(17, 1), (18, 1)]
implementation [(17, 8), (18, 8)]
implicit [(2, 25)]
In [(12, 1)]
is [(1, 11), (2, 10), (3, 8), (4, 9), (5, 6), (6, 8), (15, 5), (16, 16), (17, 23), (18, 23)]
it [(13, 67), (17, 43), (18, 43)]
let [(19, 42)]
may [(14, 19), (18, 

# With `setdefault`

In [33]:
"""Build an index mapping word -> list of occurrences"""

import sys
import re

WORD_RE = re.compile('\w+')
FILE_NAME = 'zen.txt'

index = {}

with open(FILE_NAME, encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            index.setdefault(word, []).append(location)
            
# print in alphabetical order
for word in sorted(index, key=str.upper):
    print(word, index[word])

a [(17, 48), (18, 53)]
Although [(9, 1), (14, 1), (16, 1)]
ambiguity [(12, 16)]
and [(13, 23)]
are [(19, 12)]
aren [(8, 15)]
at [(14, 38)]
bad [(17, 50)]
be [(13, 14), (14, 27), (18, 50)]
beats [(9, 23)]
Beautiful [(1, 1)]
better [(1, 14), (2, 13), (3, 11), (4, 12), (5, 9), (6, 11), (15, 8), (16, 25)]
break [(8, 40)]
cases [(8, 9)]
complex [(3, 23)]
Complex [(4, 1)]
complicated [(4, 24)]
counts [(7, 13)]
dense [(6, 23)]
do [(13, 64), (19, 48)]
Dutch [(14, 61)]
easy [(18, 26)]
enough [(8, 30)]
Errors [(10, 1)]
explain [(17, 34), (18, 34)]
Explicit [(2, 1)]
explicitly [(11, 8)]
face [(12, 8)]
first [(14, 41)]
Flat [(5, 1)]
good [(18, 55)]
great [(19, 28)]
guess [(12, 52)]
hard [(17, 26)]
honking [(19, 20)]
idea [(17, 54), (18, 60), (19, 34)]
If [(17, 1), (18, 1)]
implementation [(17, 8), (18, 8)]
implicit [(2, 25)]
In [(12, 1)]
is [(1, 11), (2, 10), (3, 8), (4, 9), (5, 6), (6, 8), (15, 5), (16, 16), (17, 23), (18, 23)]
it [(13, 67), (17, 43), (18, 43)]
let [(19, 42)]
may [(14, 19), (18, 